# Exercise 15: Text-to-Speech Conversation

**Goals**

In this notebook, you will:
- Send a short text query to GPT.
- Receive a concise AI reply.
- Convert both the user input and the AI reply into speech (TTS).
- Play back the generated audio responses.

In [27]:
from openai import OpenAI
import tempfile
import simpleaudio as sa

client = OpenAI()

In [38]:
# ----------------- Step 1: LLM Response -----------------
def chat(text_input: str) -> str:
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": "Reply in at most two concise sentences."},
            {"role": "user", "content": text_input},
        ],
        max_completion_tokens=80
    )
    return res.choices[0].message.content.strip()

In [39]:
# ----------------- Step 2: Text-to-Speech -----------------
def tts_and_play(text: str):
    if not text or not text.strip():
        return None
        
    out_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=text,
        response_format="wav",
    ) as resp:
        resp.stream_to_file(out_path)
        
    # simpleaudio
    wave_obj = sa.WaveObject.from_wave_file(out_path)
    play_obj = wave_obj.play()
    play_obj.wait_done()
    return out_path

In [40]:
# ----------------- Main -----------------
def main():
    user_text = "How do you like living in Hawaii? Have you explored mountains?"

    # 1) Display and speak the user message
    print(f"\n[You] {user_text}")
    tts_and_play(user_text)

    # 2) Generate a short reply from the LLM
    reply = chat(user_text)
    print(f"[LLM] {reply}")

    # 3) Speak the LLM reply
    tts_and_play(reply)

if __name__ == "__main__":
    main()


[You] How do you like living in Hawaii? Have you explored mountains?
[LLM] I don't have personal experiences or feelings, but Hawaii is known for its stunning landscapes and beautiful mountains, making it a popular destination for exploration and outdoor activities. Many visitors enjoy hiking in places like the Ko'olau and Waianae ranges.
